In [5]:
import pandas as pd 
from binance.client import Client

from datetime import datetime, timedelta
from prophet.serialize import model_to_json, model_from_json

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [6]:
api_key = 'Q2ScRTFxPN3auqFzSwTARZjwyReBBqvGj6o62mwHs8dE3sq6ghkrwbcmAHIIFPB9'
secret = 'YbYojfUG1Ysk1Wg6yAHt1fZmQ47Jq8fKBkIXu3JZn41XzK1G9LnJHRiIyYhx8zWH'

In [7]:
def get_wallet():
    try:
        # Essayer de lire le fichier CSV
        wallet = pd.read_csv('../out/wallet.csv', sep=';', encoding='utf-8')
        return wallet
    except FileNotFoundError as e:
        # Gérer le cas où le fichier est introuvable
        print(f"Fichier non trouvé, initialisation du portefeuille: {e}")
        wallet = pd.DataFrame({'timestamp': [datetime.now()],'fonds_disponibles': [10000], 'btc': [0], 'eth': [0], 'doge': [0], 'fonds_investis':[0], 'total_actif':[0]})
        wallet.to_csv('../out/wallet.csv', header =True, sep=';', encoding='utf-8', index=False)
        return wallet
  
def get_predict_price(symbol):
    m = load_model(symbol)
    future_df = pd.DataFrame({'ds': [datetime.now() + timedelta(days=7)]})
    predicted_value = float(m.predict(future_df).yhat.iloc[0])
    return predicted_value

def get_choice(symbol, prices):
    if get_predict_price(symbol) > float(prices.get(symbol)[0].get('price')): return 'Buy'
    elif get_predict_price(symbol) < float(prices.get(symbol)[0].get('price')): return 'Sell'

def save_model(model_prophet, symbol):
    with open('../model/serialized_model_for_'+str(symbol)+'.json', 'w') as fout:
        fout.write(model_to_json(model_prophet))  # Save model

def load_model(symbol):
    with open('../model/serialized_model_for_'+str(symbol)+'.json', 'r') as fin:
        m = model_from_json(fin.read())  # Load model
    return m



def get_current_price():
    results = {}
    interval='1d'
    symbols = ['BTCEUR', 'ETHEUR','DOGEEUR']

    # Obtenir le temps de fin (temps actuel)
    end_time = datetime.now()
    # Calculer le temps de début (1 jour dans cet exemple)
    start_time = end_time - timedelta(days=1)  # Environ 1 jour

    # Convertir start_time et end_time en millisecondes pour l'API
    start_time_str = start_time.strftime('%Y-%m-%d %H:%M:%S')
    end_time_str = end_time.strftime('%Y-%m-%d %H:%M:%S')

    for symbol in symbols:
        # Fetch historical klines data
        klines = client.get_historical_klines(symbol, interval, start_time_str, end_time_str)
        
        # Définir les colonnes pour le DataFrame
        spot_kline_cols = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

        # Convertir les données des klines en DataFrame
        df = pd.DataFrame(klines, columns=spot_kline_cols)

        # Convertir les colonnes temporelles en datetime
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')

        # Extraire le prix de fermeture et stocker dans le dictionnaire de résultats
        results[symbol] = df[['close']].rename(columns={'close': 'price'}).to_dict(orient='records')
    return results

In [75]:
def get_action(wallet, symbol, unite):
    # Obtenir le dernier index des données du portefeuille
    last_index = wallet.index[-1]
    
    # Obtenir le timestamp actuel
    current_time = datetime.now()
    
    # Obtenir les prix actuels
    prices = get_current_price()
    
    # Obtenir le choix de l'action
    action_choice = get_choice(symbol, prices)
    
    # Obtenir le prix actuel de la crypto-monnaie sélectionnée
    current_price = float(prices.get(symbol)[0].get('price'))

    # Copie de la dernière ligne et mise à jour du timestamp
    new_row = wallet.iloc[last_index].copy()
    new_row['timestamp'] = current_time

    # Mise à jour du DataFrame selon l'action choisie
    if action_choice == 'Buy':
        if new_row['fonds_disponibles'] >= unite:
            # Calcul de la quantité achetée
            amount_to_buy = unite / current_price
            new_row['fonds_disponibles'] -= unite
            new_row[symbol.lower()[:-3]] += amount_to_buy
            new_row['fonds_investis'] += unite
            print(f"Action : Acheter {amount_to_buy:.6f} {symbol} au prix de {current_price:.2f} EUR/{symbol}.")
        else:
            print("Action : Acheter impossible. Fonds insuffisants.")
    
    elif action_choice == 'Sell':
        if new_row[symbol.lower()[:-3]] >= unite:
            # Calcul de la quantité vendue et des fonds reçus
            amount_to_sell = unite
            amount_received = amount_to_sell * current_price
            new_row['fonds_disponibles'] += amount_received
            new_row[symbol.lower()[:-3]] -= amount_to_sell
            new_row['fonds_investis'] -= amount_received
            print(f"Action : Vendre {amount_to_sell:.6f} {symbol} au prix de {current_price:.2f} EUR/{symbol}.")
        else:
            print(f"Action : Vendre impossible. {symbol} insuffisants.")
    
    else:
        print("Action non reconnue. Veuillez choisir 'Buy' ou 'Sell'.")

    # Recalcule la valeur totale du portefeuille après l'action
    new_row['total_actif'] = (
        float(prices.get('BTCEUR')[0].get('price', 0)) * new_row['btc'] +
        float(prices.get('ETHEUR')[0].get('price', 0)) * new_row['eth'] +
        float(prices.get('DOGEEUR')[0].get('price', 0)) * new_row['doge'] 
    )
    
    # Ajouter la nouvelle ligne au DataFrame
    wallet = pd.concat([wallet, new_row.to_frame().T], ignore_index=True)
    return wallet

In [76]:
def update_wallet(wallet):
    # Obtenir le dernier index des données du portefeuille
    last_index = wallet.index[-1]
    last_row = wallet.iloc[last_index].copy()
    
    # Obtenir le timestamp actuel
    current_time = datetime.now()
    
    # Simule obtenir les prix actuels
    prices = get_current_price()

    # Calcule la valeur du portefeuille
    valeur_portefeuille = (
        float(prices['BTCEUR'][0]['price']) * last_row['btc'] +
        float(prices['ETHEUR'][0]['price']) * last_row['eth'] +
        float(prices['DOGEEUR'][0]['price']) * last_row['doge']
    )
    
    # Préparer une nouvelle ligne avec les valeurs mises à jour
    new_row = pd.DataFrame([{
        'timestamp': current_time,
        'fonds_disponibles': last_row['fonds_disponibles'],
        'btc': last_row['btc'],
        'eth': last_row['eth'],
        'doge': last_row['doge'],
        'fonds_investis': last_row['fonds_investis'],
        'total_actif': valeur_portefeuille
    }])
    
    # Ajouter la nouvelle ligne au DataFrame
    wallet = pd.concat([wallet, new_row], ignore_index=True)
    return wallet

In [47]:
# Initialiser 
client = Client(api_key= api_key, api_secret=secret)


In [77]:
wallet = get_wallet()  
wallet

,timestamp,fonds_disponibles,btc,eth,doge,fonds_investis,total_actif
0,2024-07-30 15:16:39.161175,10000,0,0,0,0,0


In [78]:
wallet = get_action(wallet,'BTCEUR',1000)
wallet = get_action(wallet,'ETHEUR',5000)
wallet = get_action(wallet,'DOGEEUR',500)

wallet

Action : Acheter 0.016346 BTCEUR au prix de 61178.74 EUR/BTCEUR.
Action : Acheter 1.629073 ETHEUR au prix de 3069.23 EUR/ETHEUR.
Action : Acheter 4250.255015 DOGEEUR au prix de 0.12 EUR/DOGEEUR.


,timestamp,fonds_disponibles,btc,eth,doge,fonds_investis,total_actif
0,2024-07-30 15:16:39.161175,10000,0,0,0,0,0
1,2024-07-30 16:46:11.501500,9000,0.016346,0,0,1000,1000.0
2,2024-07-30 16:46:13.927435,4000,0.016346,1.629073,0,6000,6000.0
3,2024-07-30 16:46:18.048499,3500,0.016346,1.629073,4250.255015,6500,6500.0


In [79]:
update_wallet(wallet)

,timestamp,fonds_disponibles,btc,eth,doge,fonds_investis,total_actif
0,2024-07-30 15:16:39.161175,10000,0,0,0,0,0
1,2024-07-30 16:46:11.501500,9000,0.016346,0,0,1000,1000.0
2,2024-07-30 16:46:13.927435,4000,0.016346,1.629073,0,6000,6000.0
3,2024-07-30 16:46:18.048499,3500,0.016346,1.629073,4250.255015,6500,6500.0
4,2024-07-30 16:46:32.071154,3500,0.016346,1.629073,4250.255015,6500,6500.118002
